In [1]:
#perform necessary imports

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from datetime import datetime, date, timedelta

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, LassoCV, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
df = pd.read_csv('./Desktop/temp_store/Divvy_Trips_2018_Q2.csv')

In [5]:
df.columns

Index(['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

In [ ]:
#functions for cleaning data 

def reformat_time(data_frame):

    # adding the seconds to the time
    for i in data_frame.index:

        if len(data_frame.Time[i]) == 7:
            data_frame.Time[i] = data_frame.Time[i][:4] + ':00' + ' ' + data_frame.Time[i][-2:]
        else:
            data_frame.Time[i] = data_frame.Time[i][:5] + ':00' + ' ' + data_frame.Time[i][-2:]
    
    
    # turning 12-hour to 24-hour.eed to add a method to attach the date string (2018-08-11) to the 
    # beginning of string from each time Selenium scrapes
    
    data_frame.Time = pd.to_datetime(data_frame.Time).apply(lambda x: x.strftime('%H:%M:%S'))
    
    
    
def reformat_columns(data_frame):    
    
    data_frame['Temperature'] = data_frame['Temperature'].apply(lambda x: int(x.split()[0]))
    data_frame['Dew Point'] = data_frame['Dew Point'].apply(lambda x: int(x.split()[0]))
    data_frame['Humidity'] = data_frame['Humidity'].apply(lambda x: int(x.split()[0]))
    data_frame['Wind Speed'] = data_frame['Wind Speed'].apply(lambda x: int(x.split()[0]))
    data_frame['Wind Gust'] = data_frame['Wind Gust'].apply(lambda x: int(x.split()[0]))
    data_frame['Pressure'] = data_frame['Pressure'].apply(lambda x: float(x.split()[0]))
    data_frame['Precip.'] = data_frame['Precip.'].apply(lambda x: float(x.split()[0]))
    data_frame['Precip Accum'] = data_frame['Precip Accum'].apply(lambda x: float(x.split()[0]))


    

def make_dummies(data_frame):
    
    x = pd.get_dummies(data_frame['Wind'], prefix='wind_direction', drop_first=True)
    
    y = pd.get_dummies(data_frame['Condition'], prefix='mood', drop_first=True)
    
    data_frame = data_frame.join(x)
    data_frame = data_frame.join(y)
    
    data_frame.drop(['Condition', 'Wind'], axis = 1, inplace=True)
    
    return data_frame
    

In [ ]:
#creates a list of dates for Q2 from divy data.
d1 = date(2018,4,1)
d2 = date(2018,6,30)

delta = d2 - d1

dates_list = []

for i in range(delta.days+1):
    dates_list.append(str(d1 + timedelta(i)))
    

In [ ]:
#Setting up driver to scrape data. Gives different errors depending on run, and sometimes doesn't find the table.
#Very figitty. 

chromedriver = '/Applications/chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

list_of_dfs= []
driver = webdriver.Chrome(chromedriver)

for i in dates_list:
    print(i)
    u = 'https://www.wunderground.com/history/daily/us/il/chicago/KORD/date/' + i + ".php"
    print(u)
    
    try:
        
        driver = webdriver.Chrome(chromedriver)
        driver.get(u)
        time.sleep(5)
        innerHTML = driver.execute_script("return document.body.innerHTML")

        temp = BeautifulSoup(innerHTML,'html')
        temp = temp.prettify()

        
        
        temp_almost_final = pd.read_html(temp)[1]
        
        #reformating the time
        reformat_time(temp_almost_final)
        temp_almost_final['Time'] = temp_almost_final['Time'].apply(lambda x: i + ' ' + x)
        temp_almost_final['Time'] = pd.to_datetime(temp_almost_final.Time)
        temp_almost_final['date'] = temp_almost_final.Time.apply(lambda x: x.date())
        temp_almost_final['hour'] = temp_almost_final.Time.apply(lambda x: x.hour)
        
        #cleaning up numeric columns
        reformat_columns(temp_almost_final)
        
        
        #attach the dfs to list
        list_of_dfs.append(temp_almost_final)

        #error handling functions
    except ValueError:
        print('ValueError', i)
    except IndexError:
        print('IndexError', i)
    finally:
        driver.close()
    
final_df = pd.concat(list_of_dfs, ignore_index=True)



In [ ]:
#save the final_df to Desktop
final_df.to_csv('./Desktop/final_df')

final_df = pd.read_csv('./Desktop/final_df')

#final formating of weather data.
final_df.sort_values('Time', inplace=True)
final_df.reset_index(drop=True, inplace=True)

#keeps bringing in Unnamed: 0 for some reason.
final_df.drop('Unnamed: 0', axis=1, inplace=True)
final_df.date = pd.to_datetime(final_df.date)

In [ ]:
#bringing in Divy Q2 data

divy_Q2 = pd.read_csv('./Desktop/Divvy_Trips_2018_Q2.csv')


#cleaning up and joining the two dataframes into one column
divy_Q2.start_time = pd.to_datetime(divy_Q2.start_time)

divy_Q2.sort_values('start_time', inplace=True)

divy_Q2['hour'] = divy_Q2.start_time.apply(lambda x: x.hour)
divy_Q2['date'] = pd.to_datetime(divy_Q2.start_time.apply(lambda x: x.date()))
my_ys = divy_Q2.groupby(['date', 'hour']).trip_id.count()

final_df=final_df.join(my_ys, on=['date', 'hour'], how='left')
final_df.rename(columns={'trip_id':'riders'}, inplace=True)


final_df.Time = pd.to_datetime(final_df.Time)
final_df.drop(['date', 'hour'], axis=1, inplace=True)

#use function to create dummies and save 
final_df = make_dummies(final_df)
final_final_df = final_df.to_csv('./Desktop/final_final_df')

